In [2]:
import torch
from diffusers import DDPMScheduler

scheduler = DDPMScheduler(num_train_timesteps=1000)


In [3]:
scheduler

DDPMScheduler {
  "_class_name": "DDPMScheduler",
  "_diffusers_version": "0.32.2",
  "beta_end": 0.02,
  "beta_schedule": "linear",
  "beta_start": 0.0001,
  "clip_sample": true,
  "clip_sample_range": 1.0,
  "dynamic_thresholding_ratio": 0.995,
  "num_train_timesteps": 1000,
  "prediction_type": "epsilon",
  "rescale_betas_zero_snr": false,
  "sample_max_value": 1.0,
  "steps_offset": 0,
  "thresholding": false,
  "timestep_spacing": "leading",
  "trained_betas": null,
  "variance_type": "fixed_small"
}

In [ ]:
noise_sample = scheduler.step(noise=noise, timestep=timestep, sample=sample)

In [5]:
import torch
from diffusers import FluxPipeline

# 初始化前清空缓存
torch.cuda.empty_cache()

pipe = FluxPipeline.from_pretrained(
    "/home/jiuxia/ftp_file/FLUX.1-dev",
    torch_dtype=torch.float16,
    low_cpu_mem_usage=True,
    use_safetensors=True
)

# 启用所有优化措施
pipe.enable_sequential_cpu_offload()
pipe.enable_attention_slicing(1)
pipe.enable_vae_slicing()

print(pipe.scheduler.compatibles)
pipe.load_lora_weights("dark_fantasy", weight_name="dark_fantasy_lora.safetensors", adapter_name="dark_fantasy")

prompt = "A dog holding a sign that says hello world"
image = pipe(
    prompt,
    height=256,   # 降低分辨率
    width=256,
    guidance_scale=3.5,
    num_inference_steps=50,
    max_sequence_length=128,  # 缩短序列长度
    generator=torch.Generator("cpu").manual_seed(0)
).images[0]
image.save("flux-dev-lora.png")


Loading pipeline components...: 100%|██████████| 7/7 [00:05<00:00,  1.30it/s]

[<class 'diffusers.schedulers.scheduling_flow_match_euler_discrete.FlowMatchEulerDiscreteScheduler'>]


ValueError: PEFT backend is required for this method.

: 